# IMPORT LIBRARIES

In [2]:
import pandas as pd
import numpy as np

# IMPORT DATA

In [24]:
# read the datasets
train_source = pd.read_csv('./data/training_set.csv')
test_source = pd.read_csv('./data/testing_set.csv')

In [25]:
# see the size of the datasets
train_source.shape, test_source.shape

((3509, 8), (1500, 7))

In [26]:
train_source.head()

,train_idx,Time,Open,High,Low,Close,Volume,label
0,0,2007-03-07 00:00:00,1.31258,1.31844,1.31086,1.31648,1807377.0,0
1,1,2007-03-08 00:00:00,1.31667,1.31813,1.31154,1.31396,1995920.0,1
2,2,2007-03-09 00:00:00,1.31396,1.31583,1.30864,1.31118,1859100.0,1
3,3,2007-03-11 00:00:00,1.31236,1.31415,1.31068,1.31175,192381.0,1
4,4,2007-03-12 00:00:00,1.31193,1.31992,1.31054,7.90878,2009286.0,1


In [27]:
test_source.head()

,test_idx,Time,Open,High,Low,Close,Volume
0,0,2018-05-29 00:00:00,1.16271,1.16394,1.15099,1.15343,336090
1,1,2018-05-30 00:00:00,1.15342,1.16759,1.15186,1.16627,318421
2,2,2018-05-31 00:00:00,1.16627,1.17243,1.16410,1.16938,285788
3,3,2018-06-01 00:00:00,1.16937,1.17178,1.16170,1.16584,253533
4,4,2018-06-03 00:00:00,1.16561,1.16794,1.16561,1.16757,14803


In [28]:
train_source.describe()

,train_idx,Open,High,Low,Close,Volume,label
count,3509.000000,3491.000000,3453.000000,3442.000000,3470.000000,3.481000e+03,3509.000000
mean,1754.000000,1.315917,1.308379,1.298522,1.309465,2.793843e+05,0.499858
std,1013.105375,0.463695,0.361848,0.367142,0.406838,2.995125e+05,0.500071
min,0.000000,0.136331,0.133119,0.133926,0.132743,4.970000e+02,0.000000
25%,877.000000,1.181970,1.184600,1.176797,1.181277,8.122800e+04,0.000000
50%,1754.000000,1.313140,1.318720,1.307255,1.313005,1.912090e+05,0.000000
75%,2631.000000,1.376900,1.381560,1.372110,1.376593,2.699740e+05,1.000000
max,3508.000000,12.604500,11.522790,10.871280,10.877040,2.693602e+06,1.000000


In [29]:
test_source.describe()

,test_idx,Open,High,Low,Close,Volume
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1.500000e+03
mean,749.500000,1.125877,1.129278,1.122566,1.125837,2.374014e+05
std,433.157015,0.056683,0.056129,0.057152,0.056673,2.305794e+05
min,0.000000,0.956940,0.967070,0.953570,0.956880,2.343000e+03
25%,374.750000,1.095598,1.099522,1.092465,1.095330,8.010825e+04
50%,749.500000,1.130070,1.133030,1.126825,1.129860,1.947670e+05
75%,1124.250000,1.169507,1.173942,1.166930,1.169610,3.249358e+05
max,1499.000000,1.233240,1.234940,1.226550,1.233240,2.997191e+06


In [30]:
# see the dates maximim and minimum of the train and test datasets
print("Train: ", train_source["Time"].min(), train_source["Time"].max()) 
print("Test: ", test_source["Time"].min(), test_source["Time"].max()) 

Train:  2007-03-07 00:00:00 2018-05-23 00:00:00
Test:  2018-05-29 00:00:00 2023-03-14 00:00:00


# DUPLICATES

In [31]:
# looking for duplicates
train_source.duplicated().sum()

0

In [32]:
test_source.duplicated().sum()

0

# NULL VALUES

Like we can see, there are missing values in several variables

In [33]:
# miramos si los datos tienen valores nulos
train_source.isnull().sum()

train_idx     0
Time          0
Open         18
High         56
Low          67
Close        39
Volume       28
label         0
dtype: int64

In [34]:
# calculate the percentage of null values
train_source.isnull().sum() / len(train_source) * 100

train_idx    0.000000
Time         0.000000
Open         0.512967
High         1.595896
Low          1.909376
Close        1.111428
Volume       0.797948
label        0.000000
dtype: float64

First we delete the rows with null values in the columns: "Open", "High", "Low", "Close", "Volume"

In [35]:
# delete the rows with null values in the columns: "Open", "High", "Low", "Close", "Volume"
train = train_source.dropna(subset=["Open", "High", "Low", "Close", "Volume"], how='all')

In [36]:
# however there are no rows with the previous columns with null values
train_source.shape, train.shape

((3509, 8), (3509, 8))

There could be three approaches to deal with the columns *High*, *Low*:
1. Fill the null values with the value of the previous day.
2. Fill in the null values with the total average or of x previous days.
3. Fill in the null values by adding the value of the row with the average of the variation of the last 10 days. In other words, if we have a null value on December 11, we would calculate the average of the variations of the previous 10 days. That is, if December 1 has a value of 2, December 2 has a value of 3 and so on until the 10th, we have an average variation of 1 ((1+1+1+1+ 1+1+1+1+1+1)/10). Then we would add this variation to the value of row 11.
4. Remove the null values.

In [40]:
train[["Time", "Open", "Close"]].head(10)

,Time,Open,Close
0,2007-03-07 00:00:00,1.31258,1.31648
1,2007-03-08 00:00:00,1.31667,1.31396
2,2007-03-09 00:00:00,1.31396,1.31118
3,2007-03-11 00:00:00,1.31236,1.31175
4,2007-03-12 00:00:00,1.31193,7.90878
5,2007-03-13 00:00:00,1.31811,1.31927
6,2007-03-14 00:00:00,1.31920,1.32238
7,2007-03-15 00:00:00,1.32240,1.32880
8,2007-03-16 00:00:00,1.32870,1.33106
9,2007-03-18 00:00:00,1.33064,1.33204


In [37]:
test_source.isnull().sum()

test_idx    0
Time        0
Open        0
High        0
Low         0
Close       0
Volume      0
dtype: int64

# OUTLIERS

In [14]:
# miramos si los datos tiene valores atípicos con gráficos
